### 1.简单的装饰器

In [1]:
def my_decorator(func):
    def wrapper():
        print('wrapper of decorator')
        func()
    return wrapper
def greet():
    print('hello world')
greet = my_decorator(greet)
greet()

wrapper of decorator
hello world


更简单的实现

In [2]:
def my_decorator(func):
    def wrapper():
        print('wrapper of decorator')
        func()
    return wrapper

@my_decorator
def greet():
    print("hello world")
    
greet()

wrapper of decorator
hello world


### 2.带有参数的装饰器

In [3]:
def my_decorator(func):
    def wrapper(message):
        print('wrapper of decorator')
        func(message)
    return wrapper

@my_decorator
def greet(message):
    print(message)
    
greet('hello wrold')

wrapper of decorator
hello wrold


### 3.带有自定义参数的装饰器

> 上面的装饰器的问题是参数长度固定,不具备通用性

 \*args 和\*\*kwargs,作为装饰器内部函数wrapper()的参数,<br/>
\*args和\*\*kwargs,表示接受任意数量和类型的参数

In [10]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("wrapper of decorator")
        func(*args, **kwargs)
    return wrapper

如果定义一个参数,来表示装饰器内部函数被执行的次数,那么就写为下面形式:

In [1]:
def repeat(num):
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(num):
                print('wrapper of decorator')
                func(*args,**kwargs)
        return wrapper
    return my_decorator

@repeat(4)
def greet(message):
    print(message)
    
greet("hello world")

wrapper of decorator
hello world
wrapper of decorator
hello world
wrapper of decorator
hello world
wrapper of decorator
hello world


In [2]:
greet.__name__

'wrapper'

In [3]:
help(greet)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



greet()函数被装饰后,它的元信息变了.元信息告诉我们'它不再是以前的greet()函数',而是被wrapper()函数取代了

为了解决这个问题,我们通常使用内置的装饰器@functools.wraps,它会帮助保留原函数的元信息

In [5]:
import functools

def my_decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("wrapper of decorator")
        func(*args, **kwargs)
    return wrapper

@my_decorator
def greet(message):
    print(message)
greet.__name__

'greet'

### 4.类装饰器

前面讲了函数作为装饰器的用法,实际上,类也可以作为装饰器,类装饰器主要依赖于函数\_\_call\_\_(),每当你调用一个类的示例时,函数\_\_call\_\_()就会被执行一次

In [13]:
class Count:
    def __init__(self, func):
        self.func = func
        self.num_calls = 0
    def __call__(self,*args,**kwargs):
        self.num_calls += 1
        print("num of calls is:{}".format(self.num_calls))
        return self.func(*args, **kwargs)
    
@Count
def example():
    print("hello world")


example()

num of calls is:1
hello world


In [14]:
example()

num of calls is:2
hello world


### 装饰器的嵌套

```python
@decorator1
@decorator2
@decorator3
...
```
它的执行顺序从里到外,所以上面的语句也等效于下面这行代码
```python
decorator1(decorator2(decorator3(func)))
```

In [15]:
import functools

def my_decorator1(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('execute decorator1')
        func(*args, **kwargs)
    return wrapper
    
def my_decorator2(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('execute decorator2')
        func(*args, **kwargs)
    return wrapper
    
@my_decorator1
@my_decorator2
def greet(message):
    print(message)
    
greet('hello world')

execute decorator1
execute decorator2
hello world


### 装饰器用法实例

#### 1.身份认证
大概的示例

In [16]:
import functools

def authenticate(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        request = args[0]
        if check_user_logged_in(request):
            return func(*args, **kwargs)
        else:
            raise Exception('authentication failed')
    return wrapper

@authenticate
def post_comment(request):
    pass

#### 2.日志记录

In [17]:
import time
import functools

def log_execution_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print('{} took {} ms'.format(func.__name__, (end - start) * 1000))
        return res
    return wrapper

@log_execution_time
def calculate_similarity(items):
    pass

#### 3.输入合理性检查

In [21]:
import functools
 
def validation_check(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        pass
        # 检查输入是否合法
    
@validation_check
def neural_network_training(param1, param2):
    pass


#### 4.缓存

python内置lru缓存算法,表示形式是@lru_cache

In [25]:
from functools import lru_cache


def check(x=3):
    time.sleep(3)
    print(x)
    return x+1
print(check())
print(check())

3
4
3
4


In [27]:
@lru_cache()
def check2(x=3):
    time.sleep(3)
    print(x)
    return x+1

print(check2(x=4))
print(check2())

4
5
3
4
